<a href="https://colab.research.google.com/github/katehee/Lotto_winning_number_prediction_using_LSTM/blob/master/Lotto_6_49_winning_number_prediction_using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import OneHotEncoder
from sklearn.preprocessing import LabelBinarizer

In [2]:
!wget http://www.playnow.com/resources/documents/downloadable-numbers/649.zip
!unzip 649.zip

--2022-06-16 01:21:21--  http://www.playnow.com/resources/documents/downloadable-numbers/649.zip
Resolving www.playnow.com (www.playnow.com)... 198.162.228.12
Connecting to www.playnow.com (www.playnow.com)|198.162.228.12|:80... connected.
HTTP request sent, awaiting response... 301 Moved Permanently
Location: https://www.playnow.com/resources/documents/downloadable-numbers/649.zip [following]
--2022-06-16 01:21:22--  https://www.playnow.com/resources/documents/downloadable-numbers/649.zip
Connecting to www.playnow.com (www.playnow.com)|198.162.228.12|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 46941 (46K) [application/zip]
Saving to: ‘649.zip’

649.zip             100%[===================>]  45.84K   172KB/s    in 0.3s    

2022-06-16 01:21:22 (172 KB/s) - ‘649.zip’ saved [46941/46941]

Archive:  649.zip
  inflating: 649.csv                 


In [3]:
df = pd.read_csv("649.csv")
df.head(3)

,PRODUCT,DRAW NUMBER,SEQUENCE NUMBER,DRAW DATE,NUMBER DRAWN 1,NUMBER DRAWN 2,NUMBER DRAWN 3,NUMBER DRAWN 4,NUMBER DRAWN 5,NUMBER DRAWN 6,BONUS NUMBER
0,649,1,0,1982-06-12,3,11,12,14,41,43,13
1,649,2,0,1982-06-19,8,33,36,37,39,41,9
2,649,3,0,1982-06-26,1,6,23,24,27,39,34


In [4]:
def get_winning_number(df): 
  number_set = []
  for index, row in df.iterrows(): 
    number_set.append(row[4:].tolist()) #df column number drawn 1 to Bonus 
  print("The total of",len(number_set), "draw winning numbers are retrieved")
  return number_set
winning_numbers = get_winning_number(df)

The total of 4006 draw winning numbers are retrieved


In [5]:
def number2onehot(numbers):
  encoded = np.zeros(49) 
  for number in numbers: 
    encoded[number - 1] = 1
  return encoded

def onehot2number(numbers): 
  decoded  = []
  index = 0
  for number in numbers: 
    if number == 1: 
      decoded.append(index + 1)
    index += 1 
  return decoded

enc_winning_numbers = list(map(number2onehot,winning_numbers))
# decoded_winning_numbers = list(map(onehot2number, enc_winning_numbers))

In [6]:
total_draws = len(enc_winning_numbers)
x_samples = enc_winning_numbers[0:total_draws-1]
y_samples = enc_winning_numbers[1:total_draws]

In [7]:
train_idx = (0, 3500)
val_idx = (3500, 3600)
test_idx = (3600, len(x_samples))

print("train: {0}, val: {1}, test: {2}".format(train_idx, val_idx, test_idx))

train: (0, 3500), val: (3500, 3600), test: (3600, 4005)


In [8]:
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import models

# LSTM model with 128 layers 
model = keras.Sequential([
    keras.layers.LSTM(128, batch_input_shape=(1, 1, 49), return_sequences=False, stateful=True),
    keras.layers.Dense(49, activation='sigmoid')
])

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
train_loss = []
train_acc = []
val_loss = []
val_acc = []

# max 100 epochs
for epoch in range(100):

    model.reset_states() # reset state every epoch 

    batch_train_loss = []
    batch_train_acc = []
    
    for i in range(train_idx[0], train_idx[1]):
        
        xs = x_samples[i].reshape(1, 1, 49)
        ys = y_samples[i].reshape(1, 49)
        
        loss, acc = model.train_on_batch(xs, ys) 

        batch_train_loss.append(loss)
        batch_train_acc.append(acc)

    train_loss.append(np.mean(batch_train_loss))
    train_acc.append(np.mean(batch_train_acc))

    batch_val_loss = []
    batch_val_acc = []

    for i in range(val_idx[0], val_idx[1]):

        xs = x_samples[i].reshape(1, 1, 49)
        ys = y_samples[i].reshape(1, 49)
  
        loss, acc = model.test_on_batch(xs, ys) #compare answers 
        
        batch_val_loss.append(loss)
        batch_val_acc.append(acc)

    val_loss.append(np.mean(batch_val_loss))
    val_acc.append(np.mean(batch_val_acc))

    print('epoch {0:4d} train acc {1:0.3f} loss {2:0.3f} val acc {3:0.3f} loss {4:0.3f}'.format(epoch, np.mean(batch_train_acc), np.mean(batch_train_loss), np.mean(batch_val_acc), np.mean(batch_val_loss)))


epoch    0 train acc 0.015 loss 0.416 val acc 0.020 loss 0.412
epoch    1 train acc 0.013 loss 0.412 val acc 0.010 loss 0.412
epoch    2 train acc 0.015 loss 0.410 val acc 0.010 loss 0.413
epoch    3 train acc 0.022 loss 0.409 val acc 0.020 loss 0.414
epoch    4 train acc 0.027 loss 0.406 val acc 0.040 loss 0.415
epoch    5 train acc 0.032 loss 0.403 val acc 0.040 loss 0.417
epoch    6 train acc 0.039 loss 0.400 val acc 0.030 loss 0.420
epoch    7 train acc 0.043 loss 0.397 val acc 0.030 loss 0.422
epoch    8 train acc 0.051 loss 0.393 val acc 0.020 loss 0.425
epoch    9 train acc 0.057 loss 0.387 val acc 0.010 loss 0.429
epoch   10 train acc 0.062 loss 0.382 val acc 0.010 loss 0.433
epoch   11 train acc 0.067 loss 0.375 val acc 0.010 loss 0.438
epoch   12 train acc 0.073 loss 0.368 val acc 0.010 loss 0.444
epoch   13 train acc 0.087 loss 0.360 val acc 0.010 loss 0.450
epoch   14 train acc 0.099 loss 0.352 val acc 0.000 loss 0.456
epoch   15 train acc 0.106 loss 0.345 val acc 0.000 los

In [ ]:
%matplotlib inline
import matplotlib.pyplot as plt

fig, loss_ax = plt.subplots()

acc_ax = loss_ax.twinx()

loss_ax.plot(train_loss, 'y', label='train loss')
loss_ax.plot(val_loss, 'r', label='val loss')

acc_ax.plot(train_acc, 'b', label='train acc')
acc_ax.plot(val_acc, 'g', label='val acc')

loss_ax.set_xlabel('epoch')
loss_ax.set_ylabel('loss')
acc_ax.set_ylabel('accuray')

loss_ax.legend(loc='upper left')
acc_ax.legend(loc='lower left')

plt.show()

- import past history 
- trainset include all the history to predict the latest number 
- history plot (accuracy, loss) 
- pull out most frequent number